# Loading, Extracting and Cleaning Data

In [11]:
# Importing necessary libraries

import numpy as np
import json
import re
import tensorflow as tf
import random
import spacy
nlp = spacy.load('en_core_web_sm')


In [12]:
# load file
with open('Intents_bigfile.json') as f:
    intents = json.load(f)

In [13]:
# preprocessing function
def preprocessing(line):
    line = re.sub(r'[^a-zA-z.?!\']', ' ', line)
    line = re.sub(r'[ ]+', ' ', line)
    return line

In [14]:
# extracting intent, text and response data after using the preprocessing function

inputs, targets = [], []
classes = []
intent_doc = {}

for intent in intents['intents']:
    if intent['intent'] not in classes:
        classes.append(intent['intent'])
    if intent['intent'] not in intent_doc:
        intent_doc[intent['intent']] = []
        
    for text in intent['text']:
        inputs.append(preprocessing(text))
        targets.append(intent['intent'])
        
    for response in intent['responses']:
        intent_doc[intent['intent']].append(response)

# Tokenizing input data and creating categorical targets

In [16]:
# Tokenization is a method to segregate a particular text into small chunks or tokens. 
# Here the tokens or chunks can be anything from words to characters, even subwords.
def tokenize_data(input_list):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<unk>')
    
    tokenizer.fit_on_texts(input_list)
    
    input_seq = tokenizer.texts_to_sequences(input_list)

    input_seq = tf.keras.preprocessing.sequence.pad_sequences(input_seq, padding='pre')
    
    return tokenizer, input_seq

In [17]:
# tokenize input data
tokenizer, input_tensor = tokenize_data(inputs)

In [18]:
def create_categorical_target(targets):
    word={}
    categorical_target=[]
    counter=0
    for trg in targets:
        if trg not in word:
            word[trg]=counter
            counter+=1
        categorical_target.append(word[trg])
    
    categorical_tensor = tf.keras.utils.to_categorical(categorical_target, num_classes=len(word), dtype='int32')
    return categorical_tensor, dict((v,k) for k, v in word.items())

In [19]:
# categorical target output data

target_tensor, trg_index_word = create_categorical_target(targets)

In [20]:
# How does our input and output look?
print('input shape: {} and output shape: {}'.format(input_tensor.shape, target_tensor.shape))


input shape: (143, 9) and output shape: (143, 22)


# Building model


In [22]:
# hyperparameters
epochs=60
vocab_size=len(tokenizer.word_index) + 1
embed_dim=512
units=128
target_length=target_tensor.shape[1]

In [24]:
# Recurrent Neural Network with Tensor flow

model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embed_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units, dropout=0.2)),
    tf.keras.layers.Dense(units, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(target_length, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 512)         66048     
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              656384    
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 22)                2838      
                                                                 
Total params: 758,166
Trainable params: 758,166
Non-trainable params: 0
________________________________________________

In [25]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)


In [26]:
# Training
model.fit(input_tensor, target_tensor, epochs=epochs, callbacks=[early_stop])


Epoch 1/60
5/5 [==============================] - 2s 19ms/step - loss: 3.1235 - accuracy: 0.0559
Epoch 2/60
5/5 [==============================] - 0s 19ms/step - loss: 2.7476 - accuracy: 0.2937
Epoch 3/60
5/5 [==============================] - 0s 19ms/step - loss: 1.9312 - accuracy: 0.4545
Epoch 4/60
5/5 [==============================] - 0s 20ms/step - loss: 0.9627 - accuracy: 0.7063
Epoch 5/60
5/5 [==============================] - 0s 19ms/step - loss: 0.7083 - accuracy: 0.8112
Epoch 6/60
5/5 [==============================] - 0s 20ms/step - loss: 0.2609 - accuracy: 0.9441
Epoch 7/60
5/5 [==============================] - 0s 19ms/step - loss: 0.1379 - accuracy: 0.9510
Epoch 8/60
5/5 [==============================] - 0s 18ms/step - loss: 0.1093 - accuracy: 0.9580
Epoch 9/60
5/5 [==============================] - 0s 18ms/step - loss: 0.0392 - accuracy: 0.9930
Epoch 10/60
5/5 [==============================] - 0s 17ms/step - loss: 0.0618 - accuracy: 0.9860
Epoch 11/60
5/5 [============

# The Chat Bot

In [27]:
# response function
def response(sentence):
    sent_seq = []
    doc = nlp(repr(sentence))
    
    # split the input sentences into words
    for token in doc:
        if token.text in tokenizer.word_index:
            sent_seq.append(tokenizer.word_index[token.text])

        # handle the unknown words error
        else:
            sent_seq.append(tokenizer.word_index['<unk>'])

    sent_seq = tf.expand_dims(sent_seq, 0)
    # predict the category of input sentences
    pred = model(sent_seq)

    pred_class = np.argmax(pred.numpy(), axis=1)
    
    # choice a random response for predicted sentence
    return random.choice(intent_doc[trg_index_word[pred_class[0]]]), trg_index_word[pred_class[0]]

In [ ]:
# Bot function

print("Note: Enter 'quit' to break the loop.")
while True:
    input_ = input('You: ')
    if input_.lower() == 'quit':
        break
    res, typ = response(input_)
    print('Bot: {} '.format(res))
    print()

Note: Enter 'quit' to break the loop.
You: hi
Bot: Hello, I am great, how are you? Please tell me your GeniSys user 

